In [1]:
import xml.etree.ElementTree as ET
import pandas as pd

In [2]:
with open('british-diplomats-directory.xml') as fd:
    tree = ET.parse(fd)
root = tree.getroot()


In [3]:
header_A = False
guy_dates = dict() 
person = list()
entries = list()
for child in root.iter():
    if child.attrib.get('{http://www.w3.org/XML/1998/namespace}id') == 'section-a':
        #print(child.tag, child.attrib,child.text)
        header_A = True
    if child.attrib.get('{http://www.w3.org/XML/1998/namespace}id') == 'section-b':
        header_A = False
    if header_A:
        if child.tag == "entry":
            entries.append(child)   

In [4]:
persons = list()

for entry in entries:
    for form in entry.iter("form"):
        postsDict= dict()
        for name in form.iter("persName"):
#             print(name.text)
            postsDict["persName"] = name.text
            the_name = name.text
        for date in form.iter("date"):
#             print(date.text)
            postsDict["date"] = date.text
            #person
    postsDict["seg"] = list()
    for posts in entry.iter("def"):
        for post in posts.iter("seg"):
            tmp = {"rs": list(), "dates": list()}
            for rs in post.iter("rs"):
#                 print(rs.text)
                tmp["rs"].append(rs.text)
            for date in post.iter("date"):
#                 print(date.text)
                tmp["dates"].append(date.text)
            postsDict["seg"].append(tmp)
            
        #print(postsDict)
        persons.append(postsDict)
        

In [5]:
persons[0]

{'persName': 'ABBOTT, ANTHONY J.',
 'date': '1941-',
 'seg': [{'rs': ['Consul-General, Perth, Australia'], 'dates': ['1993-1997']},
  {'rs': ['Governor of Montserrat'], 'dates': ['1997-2001']}]}

In [6]:
persons_clean = list()
for person in persons:
    ambassador = dict()
    pers_splt = person["persName"].split(",")
    ambassador["Name"] = (" ".join(pers_splt[1:])+" "+pers_splt[0]).title()
    ambassador["Date"] = person.get("date")
    ambassador["Blurb"] = ", ".join(" ".join(elt['rs'])+" "+"|".join(elt["dates"]) for elt in person['seg'])
    ambassador["persName"] = person["persName"]
    persons_clean.append(ambassador)

In [7]:
ambassadors = pd.DataFrame.from_records(persons_clean)

In [10]:
#making sure the index starts at 1
ambassadors.index = range(1,len(ambassadors)+1)

In [11]:
ambassadors = ambassadors.fillna("")

In [101]:
ambassadors.to_csv("ambassadors.csv")